In [1]:
import sys
import os
this_dir = ".."
if this_dir not in sys.path:
    sys.path.append(this_dir)

from rnn_utils import DiagnosesDataset, split_dataset, MYCOLLATE
from rnn_utils import train_one_epoch, eval_model

from mourga_variational.variational_rnn import VariationalRNN

import torch
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.model_selection import ParameterGrid, ParameterSampler

import numpy as np

import wandb

# Reproducibility

In [7]:
# Reproducibility
np.random.seed(546)
torch.manual_seed(546)
torch.cuda.manual_seed(546)

# Create dataset

In [8]:
grouping = 'ccs'
batch_size=64

In [9]:
dataset = DiagnosesDataset('../data/model_data.json',grouping)
test_size = 0.15
eval_size=0.15
eval_size_corrected = eval_size/(1-test_size)

whole_train_dataset,test_dataset = split_dataset(dataset,test_size)
train_dataset, val_dataset = split_dataset(whole_train_dataset,eval_size_corrected)

len(whole_train_dataset)
len(train_dataset)
len(val_dataset)
len(test_dataset)

whole_train_dataset = DataLoader(whole_train_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset),shuffle=True)
train_dataloader = DataLoader(train_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset),shuffle=True)#,shuffle=True)
val_dataloader = DataLoader(val_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset)) #batch_size here is arbitrary and doesn't affect total validation speed
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset))

In [10]:
next(iter(train_dataloader))['train_sequences']['original'][0]

[[129, 98, 33, 100, 101, 238, 114, 53, 2616, 60]]

# Define model

## Hyperparameters

In [14]:
input_size = next(iter(train_dataloader))['target_sequences']['sequence'].shape[2]
hidden_size = 100
num_layers = 1
n_labels = input_size
model_type = 'gru'

# Now for wandb

In [16]:
wandb.login()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## define data and train conditions

In [18]:
input_size = next(iter(train_dataloader))['target_sequences']['sequence'].shape[2]
n_labels = input_size

criterion = torch.nn.BCEWithLogitsLoss()

## Define search space

In [21]:
hyperparameters = {
    'hidden_size':[50,100,input_size], #-1 is to have the same as input size
    'num_layers':[1,2],
    'lr':[0.01,0.02],
    'model':['RNN','GRU','LSTM'],
    'dropout':[0.1,0.2],
    'batch_size':[64],
    'name':['variational']
}

meta_parameters = {
    'epochs':15
}

params = ParameterGrid(hyperparameters)
print(f'params:',len(params))

random_params = ParameterSampler(params.param_grid,n_iter=len(params)-1,random_state=231)
next(iter(random_params))

params: 72


{'num_layers': 2,
 'name': 'variational',
 'model': 'RNN',
 'lr': 0.02,
 'hidden_size': 272,
 'dropout': 0.2,
 'batch_size': 64}

## Run models

In [22]:
for param_set in params:
    config = {**param_set, 
              **meta_parameters}
    
    wandb.init(
        project="final_model_p1_pre_uncertainty_tunning", 
        config=config
    )
    
    whole_train_dataset = DataLoader(whole_train_dataset,batch_size=config['batch_size'],collate_fn=MYCOLLATE(dataset),shuffle=True)
    train_dataloader = DataLoader(train_dataset,batch_size=config['batch_size'],collate_fn=MYCOLLATE(dataset),shuffle=True)
    val_dataloader = DataLoader(val_dataset,batch_size=config['batch_size'],collate_fn=MYCOLLATE(dataset)) #batch_size here is arbitrary and doesn't affect total validation speed
    
    model = VariationalRNN(input_size=input_size,
                          hidden_size=config['hidden_size'],
                          num_layers=config['num_layers'],
                          n_labels=n_labels,
                          rnn_type=config['model'],
                          dropouti=config['dropout'],
                          dropoutw=config['dropout'],
                          dropouto=config['dropout'])
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    
    for epoch in range(1,config['epochs']+1):
        loss = train_one_epoch(model,train_dataloader,epoch,criterion,optimizer)
        wandb.log({"loss":loss})
    
    # eval model on validation data
    train_results = eval_model(model,train_dataloader,dataset, criterion, epoch, 'train_last')
    val_results = eval_model(model,val_dataloader,dataset, criterion, epoch, 'validation')
    wandb.log({'train_loss':train_results['loss'],
               'train_recall@30':train_results['last adm']['recall30']['mean'],
               'val_loss':val_results['loss'],
               'recall@10':val_results['last adm']['recall10']['mean'],
               'recall@20':val_results['last adm']['recall20']['mean'],
               'recall@30':val_results['last adm']['recall30']['mean']
              })

wandb: Currently logged in as: snovaisg (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/opt/conda/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/conda/lib/python3.8/site-packages/numpy/core/_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/opt/conda/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


loss,0.03829
_runtime,38
_timestamp,1632220471
_step,15
train_loss,0.03574
train_recall@30,0.7239
val_loss,0.0357
recall@10,0.41418
recall@20,0.58132
recall@30,0.68876


loss,█▅▃▃▂▂▂▁▁▁▁▁▁▁▁
_runtime,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆█
_timestamp,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆█
_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train_loss,▁
train_recall@30,▁
val_loss,▁
recall@10,▁
recall@20,▁
recall@30,▁


wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇█
_timestamp,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇█
_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train_loss,▁
train_recall@30,▁
val_loss,▁
recall@10,▁
recall@20,▁
recall@30,▁


wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


loss,0.02764
_runtime,57
_timestamp,1632224438
_step,15
train_loss,0.02679
train_recall@30,0.77961
val_loss,0.0287
recall@10,0.42263
recall@20,0.59031
recall@30,0.69851


loss,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁
_runtime,▁▂▂▂▃▃▄▄▄▅▅▆▆▇▇█
_timestamp,▁▂▂▂▃▃▄▄▄▅▅▆▆▇▇█
_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train_loss,▁
train_recall@30,▁
val_loss,▁
recall@10,▁
recall@20,▁
recall@30,▁


wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
